# Load dependency

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data

In [4]:
coffee_data = '../data/arabica_data_cleaned.csv'

cf = pd.read_csv(coffee_data)
cf = cf.iloc[:, 1:] # ship the number column

In [5]:
cf.head()

,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,Region,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,NaN,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,oromia,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0


In [6]:
feature_selected = [
    'Country.of.Origin', 'Variety', 
    'Processing.Method', 'Moisture', 
    'unit_of_measurement', 'Color', 
    'altitude_mean_meters', 'Total.Cup.Points'
]
cf = cf[feature_selected]
cf

,Country.of.Origin,Variety,Processing.Method,Moisture,unit_of_measurement,Color,altitude_mean_meters,Total.Cup.Points
0,Ethiopia,NaN,Washed / Wet,0.12,m,Green,2075.00,90.58
1,Ethiopia,Other,Washed / Wet,0.12,m,Green,2075.00,89.92
2,Guatemala,Bourbon,NaN,0.00,m,NaN,1700.00,89.75
3,Ethiopia,NaN,Natural / Dry,0.11,m,Green,2000.00,89.00
4,Ethiopia,Other,Washed / Wet,0.12,m,Green,2075.00,88.83
...,...,...,...,...,...,...,...,...
1306,Mexico,Bourbon,Washed / Wet,0.11,m,None,900.00,68.33
1307,Haiti,Typica,Natural / Dry,0.14,m,Blue-Green,350.00,67.92
1308,Nicaragua,Caturra,Other,0.13,m,Green,1100.00,63.08
1309,Guatemala,Catuai,Washed / Wet,0.10,ft,Green,1417.32,59.83


In [7]:
cf.columns = cf.columns.str.replace('.', '_')
cf.columns = cf.columns.str.title()
cf

/var/folders/cd/mt3mnkz168n9wr_y2d10dqb80000gn/T/ipykernel_5689/2722055113.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cf.columns = cf.columns.str.replace('.', '_')


,Country_Of_Origin,Variety,Processing_Method,Moisture,Unit_Of_Measurement,Color,Altitude_Mean_Meters,Total_Cup_Points
0,Ethiopia,NaN,Washed / Wet,0.12,m,Green,2075.00,90.58
1,Ethiopia,Other,Washed / Wet,0.12,m,Green,2075.00,89.92
2,Guatemala,Bourbon,NaN,0.00,m,NaN,1700.00,89.75
3,Ethiopia,NaN,Natural / Dry,0.11,m,Green,2000.00,89.00
4,Ethiopia,Other,Washed / Wet,0.12,m,Green,2075.00,88.83
...,...,...,...,...,...,...,...,...
1306,Mexico,Bourbon,Washed / Wet,0.11,m,None,900.00,68.33
1307,Haiti,Typica,Natural / Dry,0.14,m,Blue-Green,350.00,67.92
1308,Nicaragua,Caturra,Other,0.13,m,Green,1100.00,63.08
1309,Guatemala,Catuai,Washed / Wet,0.10,ft,Green,1417.32,59.83


In [8]:
## data cleaning for units
unit_to_convert = cf['Unit_Of_Measurement'].eq('ft')
cf.loc[unit_to_convert, ['Altitude_Mean_Meters']] /= 3.281
cf['Unit_Of_Measurement'].replace(to_replace='ft', value='m', inplace=True)
# cf.drop(columns='unit_of_measurement')

In [9]:
cf.drop(columns="Unit_Of_Measurement", inplace=True)
cf

,Country_Of_Origin,Variety,Processing_Method,Moisture,Color,Altitude_Mean_Meters,Total_Cup_Points
0,Ethiopia,NaN,Washed / Wet,0.12,Green,2075.000000,90.58
1,Ethiopia,Other,Washed / Wet,0.12,Green,2075.000000,89.92
2,Guatemala,Bourbon,NaN,0.00,NaN,1700.000000,89.75
3,Ethiopia,NaN,Natural / Dry,0.11,Green,2000.000000,89.00
4,Ethiopia,Other,Washed / Wet,0.12,Green,2075.000000,88.83
...,...,...,...,...,...,...,...
1306,Mexico,Bourbon,Washed / Wet,0.11,None,900.000000,68.33
1307,Haiti,Typica,Natural / Dry,0.14,Blue-Green,350.000000,67.92
1308,Nicaragua,Caturra,Other,0.13,Green,1100.000000,63.08
1309,Guatemala,Catuai,Washed / Wet,0.10,Green,431.978055,59.83


In [16]:
cf.dropna(inplace=True)

# split and preprocessing

In [17]:
X = cf.iloc[:, cf.columns != "Total_Cup_Points"]
y = cf["Total_Cup_Points"]
seed = 42

X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_other, y_other, test_size=0.25, shuffle=True, random_state=seed)

In [18]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(543, 6)
(181, 6)
(181, 6)


In [19]:
oh_ftr_list = ['Country_Of_Origin', 'Variety',
               'Processing_Method', 'Color']
std_ftr_list = ["Moisture", "Altitude_Mean_Meters"]

categorical_transformer = Pipeline(
    steps=
    [
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
    ]
)

numerical_transformer = Pipeline(
    steps=[
    ("scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, oh_ftr_list), 
        ('num', numerical_transformer, std_ftr_list)
    ]
)

X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)
X_val_prep = preprocessor.transform(X_val)
feature_names = preprocessor.get_feature_names_out()
feature_names

array(['cat__Country_Of_Origin_Brazil', 'cat__Country_Of_Origin_China',
       'cat__Country_Of_Origin_Colombia',
       'cat__Country_Of_Origin_Costa Rica',
       'cat__Country_Of_Origin_El Salvador',
       'cat__Country_Of_Origin_Ethiopia',
       'cat__Country_Of_Origin_Guatemala', 'cat__Country_Of_Origin_Haiti',
       'cat__Country_Of_Origin_Honduras',
       'cat__Country_Of_Origin_Indonesia', 'cat__Country_Of_Origin_Kenya',
       'cat__Country_Of_Origin_Laos', 'cat__Country_Of_Origin_Malawi',
       'cat__Country_Of_Origin_Mexico', 'cat__Country_Of_Origin_Myanmar',
       'cat__Country_Of_Origin_Nicaragua',
       'cat__Country_Of_Origin_Panama',
       'cat__Country_Of_Origin_Philippines',
       'cat__Country_Of_Origin_Rwanda', 'cat__Country_Of_Origin_Taiwan',
       'cat__Country_Of_Origin_Tanzania, United Republic Of',
       'cat__Country_Of_Origin_Thailand', 'cat__Country_Of_Origin_Uganda',
       'cat__Country_Of_Origin_United States',
       'cat__Country_Of_Origin_Un

In [20]:
X_train_prep[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.55800936,
       -0.02893808])

# model construction

In [13]:
import tensorflow as tf

In [37]:
model = tf.keras.Sequential(
    layers=[
        tf.keras.layers.Dense(units=32, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(units=32, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
#         tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(units=1)
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.mean_squared_error,
    metrics=tf.keras.metrics.mean_squared_error
)

model.fit(
    x=X_train_prep,
    y=y_train,
    batch_size=5,
    epochs=100,
    validation_data=(X_val_prep, y_val),
)

Epoch 1/100
  1/109 [..............................] - ETA: 28s - loss: 6904.7744 - mean_squared_error: 6904.7744

2023-10-28 21:31:33.163685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - 1s 6ms/step - loss: 6509.5928 - mean_squared_error: 6509.5928 - val_loss: 6233.2520 - val_mean_squared_error: 6233.2520
Epoch 2/100
 12/109 [==>...........................] - ETA: 0s - loss: 6205.7866 - mean_squared_error: 6205.7866

2023-10-28 21:31:33.881731: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - 1s 5ms/step - loss: 5718.7920 - mean_squared_error: 5718.7920 - val_loss: 5200.7695 - val_mean_squared_error: 5200.7695
Epoch 3/100
109/109 [==============================] - 1s 5ms/step - loss: 4444.2822 - mean_squared_error: 4444.2822 - val_loss: 3802.3611 - val_mean_squared_error: 3802.3611
Epoch 4/100
109/109 [==============================] - 1s 5ms/step - loss: 2999.1494 - mean_squared_error: 2999.1494 - val_loss: 2388.4070 - val_mean_squared_error: 2388.4070
Epoch 5/100
109/109 [==============================] - 1s 5ms/step - loss: 1700.7512 - mean_squared_error: 1700.7512 - val_loss: 1258.8809 - val_mean_squared_error: 1258.8809
Epoch 6/100
109/109 [==============================] - 1s 5ms/step - loss: 813.2137 - mean_squared_error: 813.2137 - val_loss: 568.3671 - val_mean_squared_error: 568.3671
Epoch 7/100
109/109 [==============================] - 1s 5ms/step - loss: 356.4262 - mean_squared_error: 356.4262 - val_loss: 253.3170 - val

Epoch 51/100
109/109 [==============================] - 1s 5ms/step - loss: 5.3719 - mean_squared_error: 5.3719 - val_loss: 11.5968 - val_mean_squared_error: 11.5968
Epoch 52/100
109/109 [==============================] - 1s 5ms/step - loss: 5.1886 - mean_squared_error: 5.1886 - val_loss: 10.7364 - val_mean_squared_error: 10.7364
Epoch 53/100
109/109 [==============================] - 1s 5ms/step - loss: 5.2823 - mean_squared_error: 5.2823 - val_loss: 12.3420 - val_mean_squared_error: 12.3420
Epoch 54/100
109/109 [==============================] - 1s 5ms/step - loss: 5.2559 - mean_squared_error: 5.2559 - val_loss: 10.7119 - val_mean_squared_error: 10.7119
Epoch 55/100
109/109 [==============================] - 1s 5ms/step - loss: 5.2731 - mean_squared_error: 5.2731 - val_loss: 12.0495 - val_mean_squared_error: 12.0495
Epoch 56/100
109/109 [==============================] - 1s 5ms/step - loss: 5.1396 - mean_squared_error: 5.1396 - val_loss: 11.0207 - val_mean_squared_error: 11.0207
Epoc

109/109 [==============================] - 1s 5ms/step - loss: 4.8744 - mean_squared_error: 4.8744 - val_loss: 10.4615 - val_mean_squared_error: 10.4615


In [38]:
pred = model.predict(X_test_prep)

6/6 [==============================] - 0s 3ms/step


2023-10-28 21:32:35.606653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [39]:
mean_squared_error(y_test, pred)

10.713781564152168

In [40]:
mean_absolute_error(y_test, pred)

2.1482072271694794